In [1]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
import albumentations
from PIL import Image
import os

# Load Label Binarizer
print("Loading label binarizer...")
try:
    lb = joblib.load("lb.pkl")
except Exception as e:
    print(f"Error loading lb.pkl: {e}")
    exit(1)

# Define Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, len(lb.classes_))
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)

try:
    model.load_state_dict(torch.load("model.pth", map_location=device))
    model.eval()
    print("Model loaded successfully...")
except Exception as e:
    print(f"Error loading model.pth: {e}")
    exit(1)

# Image Preprocessing
transform = albumentations.Compose([albumentations.Resize(224, 224)])

# Process Video
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        bbox, label, conf = cv.detect_common_objects(frame)
        isDrowning = False

        if len(bbox) == 1:
            x1, y1, x2, y2 = bbox[0]
            person_crop = frame[y1:y2, x1:x2]

            # Convert to tensor
            if person_crop.size > 0:
                with torch.no_grad():
                    pil_image = Image.fromarray(cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB))
                    pil_image = transform(image=np.array(pil_image))["image"]
                    pil_image = np.transpose(pil_image, (2, 0, 1)).astype(np.float32)
                    pil_image = torch.tensor(pil_image, dtype=torch.float).to(device).unsqueeze(0)

                    outputs = model(pil_image)
                    _, preds = torch.max(outputs.data, 1)

                predicted_label = lb.classes_[preds.item()]
                print(f"Frame {frame_count}: Detected - {predicted_label}")

                if predicted_label == "drowning":
                    isDrowning = True

        # Draw bounding boxes
        out_frame = draw_bbox(frame, bbox, label, conf, isDrowning)

        # Write to output video
        out.write(out_frame)

        frame_count += 1

    cap.release()
    out.release()
    print(f"Processing complete! Output saved as {output_path}")

# Run the function
input_video = "both2.mp4"  # Change this to your video file
output_video = "output_video.mp4"
process_video(input_video, output_video)


c:\Program Files\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Program Files\Python38\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Loading label binarizer...


c:\Program Files\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\AMALA DEVI\AppData\Local\Temp\ipykernel_19840\55640516.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals

Model loaded successfully...
Frame 13: Detected - normal
Frame 30: Detected - normal
Frame 32: Detected - normal
Frame 39: Detected - normal
Frame 40: Detected - normal
Frame 41: Detected - normal
Frame 43: Detected - normal
Frame 60: Detected - drowning



    Error 263 for command:
        open sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: sound/alarm.mp3


...

Frame 312: Detected - drowning



    Error 263 for command:
        open sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: sound/alarm.mp3
Exception in thread Thread-88:
Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\AMALA DEVI\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Drowning-Detection-\cvlib\object_detection.py", line 174, in play_sound
    playsound('sound/alarm.mp3')
  File "c:\Program Files\Python38\lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "c:\Program Files\P

Frame 313: Detected - drowning
Frame 314: Detected - normal
Frame 315: Detected - normal
Frame 316: Detected - normal



    Error 263 for command:
        open sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: sound/alarm.mp3
Exception in thread Thread-89:
Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\AMALA DEVI\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Drowning-Detection-\cvlib\object_detection.py", line 174, in play_sound
    playsound('sound/alarm.mp3')
  File "c:\Program Files\Python38\lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "c:\Program Files\P

Frame 317: Detected - drowning
Processing complete! Output saved as output_video.mp4


In [2]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
import albumentations
from PIL import Image
import os

# Load Label Binarizer
print("Loading label binarizer...")
try:
    lb = joblib.load("lb.pkl")
except Exception as e:
    print(f"Error loading lb.pkl: {e}")
    exit(1)

# Define Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, len(lb.classes_))
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)

try:
    model.load_state_dict(torch.load("model.pth", map_location=device))
    model.eval()
    print("Model loaded successfully...")
except Exception as e:
    print(f"Error loading model.pth: {e}")
    exit(1)

# Image Preprocessing
transform = albumentations.Compose([albumentations.Resize(224, 224)])

# Process Video
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        bbox, label, conf = cv.detect_common_objects(frame)
        isDrowning = False

        if len(bbox) == 1:
            x1, y1, x2, y2 = bbox[0]
            person_crop = frame[y1:y2, x1:x2]

            # Convert to tensor
            if person_crop.size > 0:
                with torch.no_grad():
                    pil_image = Image.fromarray(cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB))
                    pil_image = transform(image=np.array(pil_image))["image"]
                    pil_image = np.transpose(pil_image, (2, 0, 1)).astype(np.float32)
                    pil_image = torch.tensor(pil_image, dtype=torch.float).to(device).unsqueeze(0)

                    outputs = model(pil_image)
                    _, preds = torch.max(outputs.data, 1)

                predicted_label = lb.classes_[preds.item()]
                print(f"Frame {frame_count}: Detected - {predicted_label}")

                if predicted_label == "drowning":
                    isDrowning = True
                    print("🚨 ALERT: Drowning detected! 🚨")  # Just an alert message

        # Draw bounding boxes
        out_frame = draw_bbox(frame, bbox, label, conf, isDrowning)

        # Write to output video
        out.write(out_frame)

        frame_count += 1

    cap.release()
    out.release()
    print(f"Processing complete! Output saved as {output_path}")

# Run the function
input_video = "drowning_001.mp4"  # Change this to your video file
output_video = "output1_video.mp4"
process_video(input_video, output_video)


Loading label binarizer...


C:\Users\AMALA DEVI\AppData\Local\Temp\ipykernel_19840\1658612619.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth", map_loca

Model loaded successfully...
Frame 0: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 1: Detected - drowning
🚨 ALERT: Drowning detected! 🚨



    Error 259 for command:
        play sound/alarm.mp3 wait
    The driver cannot recognize the specified command parameter.
Exception in thread Thread-90:
Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\threading.py", line 932, in _bootstrap_inner

    Error 263 for command:
        open sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: sound/alarm.mp3


Frame 2: Detected - drowning
🚨 ALERT: Drowning detected! 🚨


...

Frame 20: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 21: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 22: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 23: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 24: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 25: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 27: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 28: Detected - drowning
🚨 ALERT: Drowning detected! 🚨
Frame 29: Detected - drowning
🚨 ALERT: Drowning detected! 🚨



    Error 259 for command:
        play sound/alarm.mp3 wait
    The driver cannot recognize the specified command parameter.


Frame 30: Detected - drowning
🚨 ALERT: Drowning detected! 🚨


Exception in thread Thread-119:
Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\threading.py", line 932, in _bootstrap_inner
    
    Error 263 for command:
        close sound/alarm.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: sound/alarm.mp3
self.run()
  File "C:\Users\AMALA DEVI\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Drowning-Detection-\cvlib\object_detection.py", line 174, in play_sound
    playsound('sound/alarm.mp3')
  File "c:\Program Files\Python38\lib\site-packages\playsound.py", line 73, in _playsoundWin
    winCommand(u'play {}{}'.format(sound, ' wait' if block else ''))
  File "c:\Program Files\Python38\lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(e

KeyboardInterrupt: 